In [1]:
import pandas as pd
import numpy as np

## Источник №1:
CSV-файл, содержащий сведения о доходах от рекламы в разбивке по моделям устройств. В файле просуммирован общий доход (revenue) по каждому устройству за май 2019 года. Каждое устройство характеризуется с помощью трех колонок: mobilePhone, mobilePhoneModel и deviceCategory.

In [2]:
#была какая-то проблема с экселем, сконвертировала через онлайн конвертер в формат csv, файл переименоменовала revenue
data = pd.read_csv('C:/Users/DariaMishina/Python Daria/revenue.csv')

In [3]:
data

,deviceCategory,mobilePhone,mobilePhoneModel,revenue
0,PC,None,None,"1883,870896034737"
1,Smartphones,ASUS,ZB500KL,"0,5209074033626201"
2,Smartphones,ASUS,ZB555KL,"0,4340029218608282"
3,Smartphones,ASUS,ZB602KL,"0,317529266551196"
4,Smartphones,ASUS,ZC554KL,"1,130402049042686"
...,...,...,...,...
141,Tablets,None,800P11B,"0,9353379155311212"
142,Tablets,Samsung,Galaxy Tab 3 7.0 Lite,"1,0139743734871542"
143,Tablets,Samsung,Galaxy Tab A 9.7,"1,0342281934243345"
144,Tablets,Samsung,Galaxy Tab E 9.6,"0,6873591515540737"


## Источник №2:
Данные Яндекс.Метрики о суточной аудитории сайта http://datalytics.ru/ в разбивке по устройствам (на этот раз данные будут настоящими). Эти данные вам необходимо получить по API с помощью токена, который вы получили при изучении модуля В10 (API Яндекс Метрики). Напоминаем, что номер счётчика, который нужно использовать для выгрузки данных — 30177909.

In [ ]:
ID: bf99db01037b493e90a0ae45de0337a3
токен AgAAAAAMknZ9AAYqN9lYMlslJUJ0l4jk3MgMMrQ

Набор группировок и метрик для получения отчета Яндекс.Метрики: 'ym:s:date', 'ym:s:deviceCategory', 'ym:s:mobilePhone', 'ym:s:mobilePhoneModel', 'ym:s:users'. Прежде чем создавать словарь с параметрами запроса вам необходимо будет определить, какие из этих параметров являются метриками, а какие — группировками.

In [ ]:
'metrics':'ym:s:users'

In [ ]:
'dimensions': 'ym:s:date,ym:s:deviceCategory,ym:s:mobilePhone,ym:s:mobilePhoneModel'

In [ ]:
params = {'metrics': 'ym:s:users',
          'dimensions': 'ym:s:date,ym:s:deviceCategory,ym:s:mobilePhone,ym:s:mobilePhoneModel',
          'date1': '2019-05-01',
          'date2': '2019-05-31',
          'limit': 100000,
          'ids': 30177909}

## Что нужно будет сделать?
- Собрать данные о доходах и данные об аудитории сайта за май 2019 года в единый отчет.
- Посчитать метрику ARPU по каждому устройству (ARPU =  revenue / суммарное количество пользователей, заходивших на сайт с данного устройства).
- Найти модель устройства с наибольшим ARPU.
В качестве ответа на финальном этапе экзамена вам нужно будет ввести название модели телефона, которой соответствует максимальное значение ARPU. Иными словами, если вы получили, например, максимальное значение ARPU для такого устройства:

deviceCategory: Smartphones

mobilePhone: Sony

mobilePhoneModel: Xperia Z,

то в качестве ответа надо будет указать название модели Xperia Z.

## Несколько важных замечаний
- При расчёте суточной аудитории необходимо учитывать только те устройства, которыми в течение месяца (напомним, что мы будем анализировать май 2019 года) воспользовались не менее 3 посетителей сайта.
- Чтобы получить общую аудиторию по конкретному устройству, необходимо суммировать количество пользователей этому устройству за каждый день.
- Поскольку задание максимально приближено к «боевым» условиям, учитывайте тот факт, что у Метрики есть ограничение по количеству запросов, и, если ваши коллеги его исчерпают, то вы можете не получить выгрузку. В этом случае попробуйте повторить обращение через некоторое время.
- Не забудьте следить за полнотой и чистотой данных в процессе выполнения задания.
- Верьте в себя и не сдавайтесь, если что-то не получится с первого раза. Помните, что вы обладаете всеми необходимыми для выполнения этого задания знаниями и навыками, и вам точно по силам справиться с ним. Мы верим в вас и ждём вас на финише!

## Задание №1
После выгрузки отчета из Яндекс.Метрики вы получили статистику суточной аудитории в разбивке по типу, модели и производителям устройств.
Чему равна сумма пользователей во всех строчках отчета из Яндекс.Метрики?

In [4]:
import json 
import requests
from pprint import pprint

In [5]:
token = 'AgAAAAAMknZ9AAYqN9lYMlslJUJ0l4jk3MgMMrQ' 
headers = {'Authorization': 'OAuth ' + token}

In [6]:
params = {'metrics': 'ym:s:users',
          'dimensions': 'ym:s:date,ym:s:deviceCategory,ym:s:mobilePhone,ym:s:mobilePhoneModel',
          'date1': '2019-05-01',
          'date2': '2019-05-31',
          'limit': 100000,
          'ids': 30177909}

In [7]:
response = requests.get('https://api-metrika.yandex.net/stat/v1/data', params=params, headers=headers)
print (response.status_code)

200


In [8]:
metrika_data = response.json()

In [9]:
print (metrika_data.keys())

dict_keys(['query', 'data', 'total_rows', 'total_rows_rounded', 'sampled', 'sample_share', 'sample_size', 'sample_space', 'data_lag', 'totals', 'min', 'max'])


In [10]:
print (metrika_data['total_rows'])

334


In [11]:
# Создаём пустой список, в который будем добавлять словари в новом формате
result = []
# Начинаем перебор элементов старого отчёта (только раздел с данными)
for data_item in metrika_data['data']:
    # Создаём словарь для хранения текущих данных в новом формате 
    new_dict = {}  
    # Обращаемся к разделу отчёта, содержащего метаданные
    # Перебираем названия группировок, которые использовались в отчёте
    for i,dimension in enumerate(data_item['dimensions']):
        # Создаём в новом словаре ключ для каждой группировки
        # Устанавливаем значение для каждого ключа
        new_dict[metrika_data['query']['dimensions'][i]] = dimension['name']
    # Те же действия выполняем для метрик
    for i,metric in enumerate(data_item['metrics']):
        new_dict[metrika_data['query']['metrics'][i]] = metric
    # Добавляем созданный словарь в итоговый список result
    result.append(new_dict)

In [12]:
import pandas as pd
df = pd.DataFrame(result)

In [13]:
df

,ym:s:date,ym:s:deviceCategory,ym:s:mobilePhone,ym:s:mobilePhoneModel,ym:s:users
0,2019-05-20,PC,None,None,109.0
1,2019-05-22,PC,None,None,103.0
2,2019-05-13,PC,None,None,97.0
3,2019-05-16,PC,None,None,96.0
4,2019-05-23,PC,None,None,93.0
...,...,...,...,...,...
329,2019-05-31,Smartphones,Samsung,Galaxy J5,1.0
330,2019-05-31,Smartphones,Huawei,STF-L09,1.0
331,2019-05-31,Smartphones,Huawei,VTR-L29,1.0
332,2019-05-31,Smartphones,Xiaomi,Redmi 5 Plus,1.0


In [14]:
df['ym:s:users'].sum()

2466.0

## Задание №2
После импорта данных по выручке из файла revenue_data.csv столбец с выручкой revenue необходимо перевести в тип float.
Какова сумма значений этого столбца? Ответ округлите до ближайшего целого числа.

In [15]:
data

,deviceCategory,mobilePhone,mobilePhoneModel,revenue
0,PC,None,None,"1883,870896034737"
1,Smartphones,ASUS,ZB500KL,"0,5209074033626201"
2,Smartphones,ASUS,ZB555KL,"0,4340029218608282"
3,Smartphones,ASUS,ZB602KL,"0,317529266551196"
4,Smartphones,ASUS,ZC554KL,"1,130402049042686"
...,...,...,...,...
141,Tablets,None,800P11B,"0,9353379155311212"
142,Tablets,Samsung,Galaxy Tab 3 7.0 Lite,"1,0139743734871542"
143,Tablets,Samsung,Galaxy Tab A 9.7,"1,0342281934243345"
144,Tablets,Samsung,Galaxy Tab E 9.6,"0,6873591515540737"


In [16]:
type(data["revenue"][0])

str

In [17]:
#делаем замену запятой на точку
data["revenue"]=data.revenue.apply(lambda x: x.replace(',', '.'))

In [18]:
#меняем тип переменной со строки на float
data["revenue"] = data["revenue"].astype(float)

In [19]:
type(data["revenue"][0])

numpy.float64

In [20]:
round(data["revenue"].sum())

2184.0

## Задание №3
Перед тем как совместить данные из отчёта Яндекс.Метрики и данные по выручке, необходимо осуществить 2 операции:

1. Сгруппировать данные из отчёта Яндекс.Метрики по 'deviceCategory', 'mobilePhone', 'mobilePhoneModel' и просуммировать количество пользователей для каждого типа устройств, посетивших сайт в течение месяца.

2. После группировки оставить только те строки получившейся таблицы, в которых оказалось 3 и более пользователей.

3. Убедитесь, что вы поработали с пропущенными (None) значениями перед тем, как делать группировку.

Сколько строк останется в датафрейме после этих действий?

In [21]:
df

,ym:s:date,ym:s:deviceCategory,ym:s:mobilePhone,ym:s:mobilePhoneModel,ym:s:users
0,2019-05-20,PC,None,None,109.0
1,2019-05-22,PC,None,None,103.0
2,2019-05-13,PC,None,None,97.0
3,2019-05-16,PC,None,None,96.0
4,2019-05-23,PC,None,None,93.0
...,...,...,...,...,...
329,2019-05-31,Smartphones,Samsung,Galaxy J5,1.0
330,2019-05-31,Smartphones,Huawei,STF-L09,1.0
331,2019-05-31,Smartphones,Huawei,VTR-L29,1.0
332,2019-05-31,Smartphones,Xiaomi,Redmi 5 Plus,1.0


In [22]:
#вместо None заполняем произвольным словом model
dfnew = df.fillna('model')
dfnew

,ym:s:date,ym:s:deviceCategory,ym:s:mobilePhone,ym:s:mobilePhoneModel,ym:s:users
0,2019-05-20,PC,model,model,109.0
1,2019-05-22,PC,model,model,103.0
2,2019-05-13,PC,model,model,97.0
3,2019-05-16,PC,model,model,96.0
4,2019-05-23,PC,model,model,93.0
...,...,...,...,...,...
329,2019-05-31,Smartphones,Samsung,Galaxy J5,1.0
330,2019-05-31,Smartphones,Huawei,STF-L09,1.0
331,2019-05-31,Smartphones,Huawei,VTR-L29,1.0
332,2019-05-31,Smartphones,Xiaomi,Redmi 5 Plus,1.0


In [23]:
#переназываем колонки чтобы быдо удобнее обращаться
dfnew.columns = ['date', 'deviceCategory', 'mobilePhone', 'mobilePhoneModel', 'users']

In [24]:
# сделать groupby по 3м категориям, а за скобки вынести users sum()
df1 = dfnew.groupby(['deviceCategory', 'mobilePhone', 'mobilePhoneModel'])['users'].sum()
df1

deviceCategory  mobilePhone  mobilePhoneModel     
PC              model        model                    2058.0
Smartphones     ASUS         ZB500KL                     1.0
                             ZB555KL                     1.0
                             ZB602KL                     1.0
                             ZC554KL                     3.0
                                                       ...  
Tablets         Samsung      Galaxy Tab 3 7.0 Lite       1.0
                             Galaxy Tab A 9.7            1.0
                             Galaxy Tab E 9.6            1.0
                Xiaomi       MI PAD 4                    1.0
                model        800P11B                     1.0
Name: users, Length: 146, dtype: float64

In [25]:
#превращаем в датафрейм
df2 = df1.reset_index()
df2

,deviceCategory,mobilePhone,mobilePhoneModel,users
0,PC,model,model,2058.0
1,Smartphones,ASUS,ZB500KL,1.0
2,Smartphones,ASUS,ZB555KL,1.0
3,Smartphones,ASUS,ZB602KL,1.0
4,Smartphones,ASUS,ZC554KL,3.0
...,...,...,...,...
141,Tablets,Samsung,Galaxy Tab 3 7.0 Lite,1.0
142,Tablets,Samsung,Galaxy Tab A 9.7,1.0
143,Tablets,Samsung,Galaxy Tab E 9.6,1.0
144,Tablets,Xiaomi,MI PAD 4,1.0


In [26]:
#сделать фильтр по колонке где больше или равно 3м пользователям
df3 = df2[df2.users >= 3]
df3

,deviceCategory,mobilePhone,mobilePhoneModel,users
0,PC,model,model,2058.0
4,Smartphones,ASUS,ZC554KL,3.0
8,Smartphones,Apple,iPhone,125.0
14,Smartphones,Huawei,AUM-L29,3.0
17,Smartphones,Huawei,CLT-L29,3.0
18,Smartphones,Huawei,COL-L29,3.0
30,Smartphones,Huawei,JSN-L21,3.0
31,Smartphones,Huawei,LLD-L31,5.0
34,Smartphones,Huawei,STF-L09,5.0
47,Smartphones,Meizu,16th,3.0


In [27]:
len(df3)

34

## Задание №4
Теперь нужно посчитать значение ARPU для каждого устройства. Для этого вам нужно объединить отчёт Яндекс.Метрики с данными из файла с выручкой и посчитать значение ARPU (revenue / users) для каждой строки получившегося датафрейма.

Какое значение ARPU получится у строки с категорией устройства 'PC' (модель и тип устройства при этом будут не определены)? Ответ округлите до двух знаков после запятой. Формат ответа: 0.84

In [28]:
#отчёт Яндекс.Метрики 
df3

,deviceCategory,mobilePhone,mobilePhoneModel,users
0,PC,model,model,2058.0
4,Smartphones,ASUS,ZC554KL,3.0
8,Smartphones,Apple,iPhone,125.0
14,Smartphones,Huawei,AUM-L29,3.0
17,Smartphones,Huawei,CLT-L29,3.0
18,Smartphones,Huawei,COL-L29,3.0
30,Smartphones,Huawei,JSN-L21,3.0
31,Smartphones,Huawei,LLD-L31,5.0
34,Smartphones,Huawei,STF-L09,5.0
47,Smartphones,Meizu,16th,3.0


In [29]:
#данные из файла с выручкой
data

,deviceCategory,mobilePhone,mobilePhoneModel,revenue
0,PC,None,None,1883.870896
1,Smartphones,ASUS,ZB500KL,0.520907
2,Smartphones,ASUS,ZB555KL,0.434003
3,Smartphones,ASUS,ZB602KL,0.317529
4,Smartphones,ASUS,ZC554KL,1.130402
...,...,...,...,...
141,Tablets,None,800P11B,0.935338
142,Tablets,Samsung,Galaxy Tab 3 7.0 Lite,1.013974
143,Tablets,Samsung,Galaxy Tab A 9.7,1.034228
144,Tablets,Samsung,Galaxy Tab E 9.6,0.687359


In [31]:
#merge по колонке deviceCategory дает в итоге неверный результат, такой способ подходит только для РС, перемэппливается все со всем, эппл и самсунг, категория устройства-то одна - смартфон
joined = df3.merge(data, on='deviceCategory', how='left')

In [33]:
#добавляем новую колонку с ARPU
joined['ARPU'] = joined['revenue'] / joined['users']

In [34]:
joined.head()

,deviceCategory,mobilePhone_x,mobilePhoneModel_x,users,mobilePhone_y,mobilePhoneModel_y,revenue,ARPU
0,PC,model,model,2058.0,None,None,1883.870896,0.915389
1,Smartphones,ASUS,ZC554KL,3.0,ASUS,ZB500KL,0.520907,0.173636
2,Smartphones,ASUS,ZC554KL,3.0,ASUS,ZB555KL,0.434003,0.144668
3,Smartphones,ASUS,ZC554KL,3.0,ASUS,ZB602KL,0.317529,0.105843
4,Smartphones,ASUS,ZC554KL,3.0,ASUS,ZC554KL,1.130402,0.376801


In [37]:
round(joined.ARPU[0],2)

0.92

## Итоговое задание
Итак, финальный вопрос: какая модель устройства mobilePhoneModel имеет максимальный ARPU? Введите название модели БЕЗ кавычек или апострофов.

In [44]:
data

,deviceCategory,mobilePhone,mobilePhoneModel,revenue
0,PC,None,None,1883.870896
1,Smartphones,ASUS,ZB500KL,0.520907
2,Smartphones,ASUS,ZB555KL,0.434003
3,Smartphones,ASUS,ZB602KL,0.317529
4,Smartphones,ASUS,ZC554KL,1.130402
...,...,...,...,...
141,Tablets,None,800P11B,0.935338
142,Tablets,Samsung,Galaxy Tab 3 7.0 Lite,1.013974
143,Tablets,Samsung,Galaxy Tab A 9.7,1.034228
144,Tablets,Samsung,Galaxy Tab E 9.6,0.687359


In [48]:
data[data.mobilePhoneModel == 'iPhone']

,deviceCategory,mobilePhone,mobilePhoneModel,revenue
8,Smartphones,Apple,iPhone,97.972879
9,Smartphones,Beeline,iPhone,0.670827
46,Smartphones,Megafon,iPhone,0.693217
100,Smartphones,Tele2,iPhone,0.808822


In [41]:
joined1 = df3.merge(data, on='mobilePhoneModel', how='left')

In [42]:
joined1

,deviceCategory_x,mobilePhone_x,mobilePhoneModel,users,deviceCategory_y,mobilePhone_y,revenue
0,PC,model,model,2058.0,NaN,NaN,NaN
1,Smartphones,ASUS,ZC554KL,3.0,Smartphones,ASUS,1.130402
2,Smartphones,Apple,iPhone,125.0,Smartphones,Apple,97.972879
3,Smartphones,Apple,iPhone,125.0,Smartphones,Beeline,0.670827
4,Smartphones,Apple,iPhone,125.0,Smartphones,Megafon,0.693217
5,Smartphones,Apple,iPhone,125.0,Smartphones,Tele2,0.808822
6,Smartphones,Huawei,AUM-L29,3.0,Smartphones,Huawei,2.388569
7,Smartphones,Huawei,CLT-L29,3.0,Smartphones,Huawei,3.470900
8,Smartphones,Huawei,COL-L29,3.0,Smartphones,Huawei,2.758386
9,Smartphones,Huawei,JSN-L21,3.0,Smartphones,Huawei,3.347500


In [49]:
joined1['ARPU'] = joined1['revenue'] / joined1['users']

In [50]:
joined1

,deviceCategory_x,mobilePhone_x,mobilePhoneModel,users,deviceCategory_y,mobilePhone_y,revenue,ARPU
0,PC,model,model,2058.0,NaN,NaN,NaN,NaN
1,Smartphones,ASUS,ZC554KL,3.0,Smartphones,ASUS,1.130402,0.376801
2,Smartphones,Apple,iPhone,125.0,Smartphones,Apple,97.972879,0.783783
3,Smartphones,Apple,iPhone,125.0,Smartphones,Beeline,0.670827,0.005367
4,Smartphones,Apple,iPhone,125.0,Smartphones,Megafon,0.693217,0.005546
5,Smartphones,Apple,iPhone,125.0,Smartphones,Tele2,0.808822,0.006471
6,Smartphones,Huawei,AUM-L29,3.0,Smartphones,Huawei,2.388569,0.796190
7,Smartphones,Huawei,CLT-L29,3.0,Smartphones,Huawei,3.470900,1.156967
8,Smartphones,Huawei,COL-L29,3.0,Smartphones,Huawei,2.758386,0.919462
9,Smartphones,Huawei,JSN-L21,3.0,Smartphones,Huawei,3.347500,1.115833


In [51]:
joined1.ARPU.max()

1.1569667733810385

In [52]:
joined1.mobilePhoneModel[joined1.ARPU.idxmax()]

'CLT-L29'